In [ ]:
import panel as pn
from ipyleaflet import Map, basemaps, DrawControl
pn.extension()

In [ ]:
# Create widgets.
headings = pn.pane.Markdown('''
## Approximately how many seconds have you lived?
### Enter your information here and see!
''')
name = pn.widgets.TextInput(name='Name', value='Venus')
age = pn.widgets.IntSlider(name='Age', value=50, start=1, end=100)
color = pn.widgets.ColorPicker(name='Color', value='#4f4fdf')
button = pn.widgets.Button(name='Calculate', button_type='primary')
result = pn.widgets.StaticText()

# Create functions.
def compute_secs_lived(event):
  secs = age.value * 365 * 24 * 60 * 60
  result.value = f'Hello {name.value}, you have lived for approximately {secs} seconds!'
  result.style = {'color': color.value}
  return result

# Create interactions between widgets and functions.
button.on_click(compute_secs_lived)

In [ ]:
# Create map.
map = Map(basemap=basemaps.OpenStreetMap.France, center=[38.128, 2.588], zoom=5)

# Create drawing controls.
draw_control = DrawControl()
draw_control.rectangle = {
  "shapeOptions": {
    "fillColor": "#4f4fdf",
    "color": "#4f4fdf",
    "fillOpacity": 0.3
  }
}
map.add_control(draw_control)

In [ ]:
# Add components to serve on app.
app = pn.Column("# Testing Panel", headings, name, age, color, button, result, map)
app.servable()